In [65]:
from numpy import *
from functools import reduce
import re
import os
import random
import jieba
from itertools import chain
import math
import operator

In [66]:
# 保留中文内容
# 爬虫下来的文档中会有冗余信息
def find_chinese(file):
    pattern = re.compile(r'[^\u4e00-\u9fa5]')
    chinese = re.sub(pattern, '', file)
    return chinese

In [67]:
# 准备数据：从文本中构建词向量
def loadDataSet2():
    # 0 --> 党政办公室
    # 1 --> 教务部
    # 2 --> 招生办公室
    # 3 --> 研究生院
    # 4 --> 科学技术部
    postingList = []
    classVec = []
    for file in os.listdir(f'data/'):
        text = open(f'data/{file}', 'r').read()
        text = find_chinese(text)
        
        text =  list(jieba.cut(text, cut_all=False))
        postingList.append(text)

        if file.startswith('党政办公室'):
            classVec.append(0)
        elif file.startswith('教务部'):
            classVec.append(1)
        elif file.startswith('招生办公室'):
            classVec.append(2)
        elif file.startswith('研究生院'):
            classVec.append(3)
        elif file.startswith('科学技术部'):
            classVec.append(4)
            
    return postingList, classVec

In [68]:
# 准备数据：从文本中构建词向量
def loadDataSet(goal):
    # 0 --> 党政办公室
    # 1 --> 教务部
    # 2 --> 招生办公室
    # 3 --> 研究生院
    # 4 --> 科学技术部
    postingList = []
    classVec = []
    for file in os.listdir(f'data1/{goal}'):
        text = open(f'/Users/alex_shen/SynologyDrive/PcBackup/深圳大学/课程/大三下/信息检索/实验/202102-信息检索-HW5/data1/{goal}/{file}', 'r').read()
        text = find_chinese(text)
        
        text =  list(jieba.cut(text, cut_all=False))
        postingList.append(text)

        if file.startswith('党政办公室'):
            classVec.append(0)
        elif file.startswith('教务部'):
            classVec.append(1)
        elif file.startswith('招生办公室'):
            classVec.append(2)
        elif file.startswith('研究生院'):
            classVec.append(3)
        elif file.startswith('科学技术部'):
            classVec.append(4)
            
    return postingList, classVec

In [69]:
# 构建一个包含所有文档词汇且不重复的词汇表
def createVocabList(dataSet):
    vocabSet = set([])  # 创建一个空集
    for document in dataSet:
        vocabSet = vocabSet | set(document)  # 创建两个集合的并集
    return list(vocabSet)

In [70]:
def concatenateTextOfAllDocsInClass(postingList, classVec, c):
    text_c = []
    for index, cat in enumerate(classVec):
        if cat == c:
            text_c.extend(postingList[index])
    return text_c

In [71]:
def trainNB(myVocabList, postingList, classVec):
    V = list(chain(*myVocabList))
    N = len(postingList)
    condprob = {term: [0] * 5 for term in V}
    prior = [0] * 5
    for c in range(5):
        Nc = classVec.count(c)
        prior[c] = Nc / N
        text_c = concatenateTextOfAllDocsInClass(postingList, classVec, c)
        for t in V:
            T_ct = text_c.count(t)
            condprob[t][c] = (T_ct + 1) / (len(text_c) + len(V))
    return V, prior, condprob

In [72]:
def applyMultinomialNB(V, prior, condprob, text):
    p = []
    for c in range(5):
        p.append(math.log(prior[c], 2))
        for t in text:
            if t in V:
                p[c] += math.log(condprob[t][c], 2)
    return p.index(max(p))

In [73]:
def readFIles(files: list(list())):
    # 读取data文件夹下的所有文件
    for file in os.listdir('data'):
        text = open(f'data/{file}', 'r').read()
        text = find_chinese(text)
        
        # 根据文件名进行分类
        if file.startswith('党政办公室'):
            files[0].append(text)
        elif file.startswith('教务部'):
            files[1].append(text)
        elif file.startswith('招生办公室'):
            files[2].append(text)
        elif file.startswith('研究生院'):
            files[3].append(text)
        elif file.startswith('科学技术部'):
            files[4].append(text)

# 利用jieba进行分析
def cutWords(files: list(list()), terms: list(list()), term_list: set()):
    for index, file in enumerate(files):
        for text in file:
            words = list(jieba.cut(text, cut_all=False))
            terms[index].append(words)
            term_list.update(set(words))
            
# 计算MI值
def cal_MI(N11, N01, N10, N00):
    N = N11+N01+N10+N00
    sum = 0
    sum += 0 if N11 == 0 else N11/N*log(N*(N11)/(N11+N10)/(N11+N01), 2)
    sum += 0 if N01 == 0 else N01/N*log(N*N01/(N01+N00)/(N11+N01), 2)
    sum += 0 if N10 == 0 else N10/N*log(N*N10/(N10+N11)/(N10+N00), 2)
    sum += 0 if N00 == 0 else N00/N*log(N*N00/(N00+N01)/(N00+N10), 2)
    return sum
# 计算X^2值
def cal_X2(N11, N01, N10, N00):
    sum = (N11+N10+N01+N00)*((N11*N00-N10*N01)**2)/(N11+N01+1)/(N10+N00+1)/(N11+N10+1)/(N01+N00+1)
    return sum

# 重建小顶堆
def sift(li,low,high):
    i = low
    #找孩子 左孩子
    j = 2 * i +1
    tmp = li[low] #把堆顶存起来
    while  j <= high: #只要j位置有数
        #右孩子要右，右孩子比较大 并且右孩子大于左孩子     右孩子不越界 j+1 <=hight 
        if j+1 <=high  and li[j+1][1] < li[j][1]:
        #j 指向右孩子  这个指的是两个孩子更大的数
            j = j + 1
        if li[j][1] < tmp[1]:  #目前要是,tmp大就放过去,还是j大放上面去
            li[i] = li[j]
            i = j       #可以交换了    现在j等于新的i     i等于原来的j
            j = 2 * i + 1
        else :  #tmp 更大,把tmp放到i的位置上  假如堆点是6
            li[i] = tmp  # 6放到原来8的
            break
    else: #就是2没法和别的比了,就放到该去的地方去
        li[i] = tmp  #把tmp放到叶子节点上去

#最后非叶子节点
#n的最后一个下标是n-1,找他的父亲就孩子,(n-1-1)/2
#孩子下标是 n
#孩子找父亲 (n-1)/2 里面的n的最后一个下标是n-1 所有(n-1-1)/2 -> (n-2)/2

def topk(li,k):
    # 前K元素建立堆
    heap = li[0:k]
    for i in range((k-2)//2,-1,-1):
        sift(heap,i,k-1)

    #1.建堆
    for i in range(k,len(li)):
        if li[i][1] > heap[0][1]:
            heap[0] = li[i]
            sift(heap,0,k-1)

    #2. 堆排序
    for i in range(k-1,-1,-1):
        heap[0], heap[i] = heap[i], heap[0]
        sift(heap,0,i -1)
    return [word[0] for word in heap]

In [74]:
def get_X_2():
    term_list = set()
    # 0 --> 党政办公室
    # 1 --> 教务部
    # 2 --> 招生办公室
    # 3 --> 研究生院
    # 4 --> 科学技术部
    files = [[] for i in range(5)]
    terms = [[] for i in range(5)]

    # 依次打开data文件夹下的文件
    readFIles(files)

    # 进行中文分词，并生成词典
    cutWords(files, terms, term_list)

    # 计算每个单词的互信息
    MI = [{} for i in range(5)]
    X2 = [{} for i in range(5)]
    for word in term_list:
        cat_t, cat_f = [[] for i in range(5)], [[] for i in range(5)]

        # 统计每一个category中的出现情况
        # cat_t[i] 表示第i个category中出现的word的doc的个数
        # cat_f[i] 表示第i个category中不出现的word的doc的个数
        for index in range(5):
            cat_t[index] = list(
                word in words for words in terms[index]).count(True)
            cat_f[index] = list(
                word in words for words in terms[index]).count(False)

        # 计算混淆矩阵
        for index in range(5):
            N11 = cat_t[index]
            N01 = cat_f[index]
            N10 = sum(cat_t)-N11
            N00 = sum(cat_f) - N01
            # 计算MI值
            socre2 = cal_X2(N11, N01, N10, N00)

            X2[index][word] = socre2

    res = []
    for i in range(5):
        res.append(topk(list(X2[i].items()), 15))

    return res


In [75]:
if __name__ == '__main__':
    postingList, classVec = loadDataSet('train')
    myVocabList = get_X_2();
    
    
    V, prior, condprob = trainNB(myVocabList, postingList, classVec)
    
    test_postingList, test_classVec = loadDataSet('test')
    
    correct = [0] * 5
    for index, text in enumerate(test_postingList):
        res = applyMultinomialNB(V, prior, condprob, text)
        if res == test_classVec[index]:
            correct[test_classVec[index]] += 1
    
    # 0 --> 党政办公室
    # 1 --> 教务部
    # 2 --> 招生办公室
    # 3 --> 研究生院
    # 4 --> 科学技术部
    print('整体 正确率：', sum(correct) / len(test_postingList))
    print('党政办公室 正确率：', correct[0] / test_classVec.count(0))
    print('教务部 正确率：', correct[1] / test_classVec.count(1))
    print('招生办公室 正确率：', correct[2] / test_classVec.count(2))
    print('研究生院 正确率：', correct[3] / test_classVec.count(3))
    print('科学技术部 正确率：', correct[4] / test_classVec.count(4))
    

整体 正确率： 0.94
党政办公室 正确率： 1.0
教务部 正确率： 0.7
招生办公室 正确率： 1.0
研究生院 正确率： 1.0
科学技术部 正确率： 1.0
